In [1]:
# importing required modules here
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import chardet as cd
import PyPDF2


In [2]:
def find_encoding(file):
    return cd.detect(open(file, 'rb').read())['encoding']

In [3]:
encoding_for_companies_file = find_encoding('companies.txt')
encoding_for_rounds_file  = find_encoding('rounds2.csv')
encoding_for_mapping_file  = find_encoding('mapping.csv')

In [4]:
companies = pd.read_csv('companies.txt',sep='\t',encoding=encoding_for_companies_file)
rounds2 = pd.read_csv('rounds2.csv',encoding = encoding_for_rounds_file)
mapping = pd.read_csv('mapping.csv',encoding = encoding_for_mapping_file)

In [5]:
# rounds2.drop_duplicates(subset='company_permalink',inplace=True)
companies['permalink'] = companies['permalink'].str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()

rounds2.rename(columns={'company_permalink': 'permalink'}, inplace=True)

In [6]:
# Checkpoint 1
print ('Number of unique companies in rounds2: ',rounds2['permalink'].unique().shape)
print ('Number of unique companies in companies: ',companies['permalink'].unique().shape)

master_frame = pd.merge(companies,rounds2,on='permalink',how='inner')
print ('number of companies in combined dataframe: ',master_frame.shape[0])

Number of unique companies in rounds2:  (66370,)
Number of unique companies in companies:  (66368,)
number of companies in combined dataframe:  114942


In [7]:
master_frame.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [8]:
# Checkpoint 2


frt = master_frame.groupby('funding_round_type')
print (frt['raised_amount_usd'].mean().sort_values(ascending = False))
master_frame['funding_round_type'].str.lower()
master_frame = master_frame[master_frame['funding_round_type'] == 'venture']
# master_frame.funding_round_type
master_frame.head()

funding_round_type
post_ipo_debt            1.687046e+08
post_ipo_equity          8.218249e+07
secondary_market         7.964963e+07
private_equity           7.330859e+07
undisclosed              1.924237e+07
debt_financing           1.704353e+07
venture                  1.174895e+07
grant                    4.300576e+06
convertible_note         1.453439e+06
product_crowdfunding     1.363131e+06
angel                    9.586945e+05
seed                     7.197796e+05
equity_crowdfunding      5.383682e+05
non_equity_assistance    4.112031e+05
Name: raised_amount_usd, dtype: float64


,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0
5,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN


In [37]:
# Checkpoint 3
master_frame['country_code'].unique()

country_wise_investment = master_frame.groupby('country_code')
top9 = pd.DataFrame(country_wise_investment['raised_amount_usd'].sum().sort_values(ascending = False)[:9])
top9.head()
print (top9.sort_values(by='raised_amount_usd',ascending = False))

              raised_amount_usd
country_code                   
USA                4.225108e+11
CHN                3.983542e+10
GBR                2.024563e+10
IND                1.439186e+10
CAN                9.583332e+09
FRA                7.259537e+09
ISR                6.907515e+09
DEU                6.346960e+09
JPN                3.363677e+09


In [124]:
# Checkpoint 4

# print(master_frame['category_list'])

print(list(mapping))
master_frame['primary_sector'] = pd.Series([str(i).split('|')[0] for i in master_frame['category_list']])
# print(master_frame.head())
# master_frame['main_sector'] = master_frame[]
main_sector = {}
for i in mapping.index:
    main_sector[mapping[mapping.columns.values[0]][i]] = (mapping.loc[i][mapping.loc[i] == 1].index[0])
print(main_sector)

master_frame['primary_sector']

# print(pd.dataframe(frt))
# Total number of Investments (count)

['category_list', 'Automotive & Sports', 'Blanks', 'Cleantech / Semiconductors', 'Entertainment', 'Health', 'Manufacturing', 'News, Search and Messaging', 'Others', 'Social, Finance, Analytics, Advertising']


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy [ipykernel_launcher.py:6]


{nan: 'Blanks', 'Information Technology': 'Social, Finance, Analytics, Advertising', 'Open Source': 'Others', 'Indoor Positioning': 'Others', 'Brewing': 'Entertainment', 'Clean Technology IT': 'Cleantech / Semiconductors', 'Logistics': 'Others', 'E-Commerce Platforms': 'Others', 'Social + Mobile + Local': 'Social, Finance, Analytics, Advertising', 'All Markets': 'Others', 'Debt Collecting': 'Social, Finance, Analytics, Advertising', 'Technology': 'Others', 'Distribution': 'Others', 'Recreation': 'Others', 'Fi0nce': 'Social, Finance, Analytics, Advertising', 'Call Center Automation': 'News, Search and Messaging', 'Solar': 'Cleantech / Semiconductors', 'Web Browsers': 'News, Search and Messaging', 'Law Enforcement': 'Others', 'Events': 'Entertainment', 'Computers': 'Manufacturing', 'Mobile': 'News, Search and Messaging', 'Productivity Software': 'Others', 'VoIP': 'Entertainment', 'Customer Support Tools': 'Social, Finance, Analytics, Advertising', 'Virtual Workforces': 'Entertainment', '

0                         Media
1         Application Platforms
3                   Curated Web
4                      Software
5                 Biotechnology
8                     Analytics
10                    Education
11                       Beauty
12                  Curated Web
22                        Games
28                  Health Care
33                Biotechnology
34                Social Travel
38                          nan
39                     Internet
41                       Mobile
42                Manufacturing
44                Manufacturing
46                Manufacturing
47                Manufacturing
49                Manufacturing
50                   E-Commerce
52                  Advertising
53                     Software
54        Public Transportation
55                          nan
56                     Software
57                     Internet
59                       Travel
61                    Education
                  ...          
114884  

In [126]:
master_frame.to_csv('out.csv')